In [1]:
import os
%run funciones.py
import json
import pandas as pd
import numpy as np
from funciones import *
from keras.models import load_model

In [2]:
def predecir1(x, model, scaler, conv):
    # Calcular predicción escalada en el rango de -1 a 1
    y_pred_s = model.predict(x,verbose=1)
    
    if conv:
       y_pred_s = y_pred_s[:, 0, 0, 0].reshape((-1, 1))

    # Llevar la predicción a la escala original
    y_pred = scaler.inverse_transform(y_pred_s)

    return y_pred.flatten()

In [3]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

def RMSE_MSE_VAL_SEC(Real_vec, Pred_vec):
    vector_real = Real_vec.flatten()
    vector_predicho = Pred_vec

    # Calcular las métricas RSME - MSE - MAE
    rmse = np.sqrt(mean_squared_error(vector_real, vector_predicho))
    mae = mean_absolute_error(vector_real, vector_predicho)
    mape = mean_absolute_percentage_error(vector_real, vector_predicho)

    # Calcular el RMSE
    return rmse, mae, mape

In [5]:
pruebas = pd.read_csv("..\data\pruebas.csv")

stop = list(pruebas['name stops'])
real = list(pruebas['real'])
link = list(zip(pruebas['ini_link'], pruebas['end_link']))
time = list(zip(pruebas['hora'], pruebas['minuto']))

c = ['Nro','Stops','Link', 'Time', 'Real', 'Mean_Set', 'err_mean_set', 'input_set', 'err_input_set', 'input_concat', 'err_input_concat', 'model']
preds = pd.DataFrame(columns=c)

for file in os.listdir('../models/NVIDIA GeForce RTX 3070 Ti Laptop GPU/'):
    model_path = f'../models/NVIDIA GeForce RTX 3070 Ti Laptop GPU/{file}'
    json_path = f'../info_models/NVIDIA GeForce RTX 3070 Ti Laptop GPU/{file[:-2]}json'

    # cargar archivos
    info_model = json.load(open(json_path))
    model = load_model(model_path, compile=False)

    # ingresar parametros
    div_split = info_model['div_split']
    input_out_length = (info_model['INPUT_LENGTH'], info_model['OUTPUT_LENGTH'])  
    SCALER = info_model['scaler']
    CONV = info_model['isCONV']
    range_links = tuple(info_model['range_links'])
    path_dataset = "../data/"+info_model['name_dataset']
    input_shape = (1, ) + model.input_shape[1:]
    print(f"{file}")
    print(f"Parametros de prediccion: {div_split}, {input_out_length}, {SCALER}, CONV:{CONV}, {range_links}, {path_dataset}, {input_shape}")

    # leer dataset
    cols = ['id_linkref', 'horas', 'minute', 'travel_time']
    origin_dataset = pd.read_csv(path_dataset, sep=',', usecols=cols)
    dataset = origin_dataset[cols]

    # delimitar links
    if range_links != (0, 82):
        dataset =  dataset[(dataset['id_linkref']>= range_links[0]) & (dataset['id_linkref']<=range_links[1])]
    print(f"Links del dataset desde {range_links[0]} - {range_links[1]} = {len(dataset['id_linkref'].unique())} links")

    # preparar dataset para la prediccion segun el modelo
    data, scaler = cargar_dataset(dataset, div_split, input_out_length, SCALER, CONV)

    # cacular RMSE datos no escalados
    # y_tr_pred = predecir(x_tr_s, model, SCALER)
    #y_ts_pred = predecir1(data['x_ts_s'], model, scaler, conv=CONV)
    #rr = RMSE_MSE_VAL_SEC(data['y_ts'], y_ts_pred)
    #print(rr)
    # y_vl_pred = predecir(x_vl_s, model, SCALER)

    dt_input = data['ts'].reset_index()
    for i in range(len(real)):
        # encontrar input: retorna indices de los samples que coniden como input
        res = find_input(dt_input, stop=link[i][0], time=time[i], input_size=input_out_length[0])

        # escoger un indice aleatorio para pa prediccion
        #if len(res)==0:
        #    print(f'sample {i} sin input coincidente')
        #    continue
        tmp = np.random.choice(len(res))

        # predecir el tiempo de viaje
        mean, y_of_set, y_of_pred = predict_travel_time(
            n_links = link[i][1] - link[i][0],
            shape_input= input_shape,
            id_sample= res[tmp],
            data_in_s= data['x_ts_s'],
            data_in = data['x_ts'],
            modelo = model,
            scaler=scaler,
            conv=CONV
        )

        # calcular tiempo de viaje, suma de todas las predicciones en cada link
        p_mean = round(sum(mean),0) # promedio, para verificar en que se basa las predicciones
        p_set = round(sum(y_of_set), 0) # usamos input un grupo de muestras del dataset test
        p_concat = round(sum(y_of_pred), 0) # concadenamos las predicciones al siguiente input

        # calcular el error de cada forma de prediccion
        err_p_mean = real[i] - p_mean
        err_p_set = real[i] - p_set
        err_p_concat = real[i] - p_concat 

        # agregar al df pred
        preds.loc[len(preds)] = [i+1, stop[i], link[i], time[i], real[i], p_mean, err_p_mean, p_set, err_p_set, p_concat, err_p_concat, file[:-3]]

    print('=> Pruebas Completadas')
    print('='*120)

AUTOENCODER-LSTM_e200b256_Robust_in83-4_dt0-82.h5
Parametros de prediccion: 83, (83, 1), Robust, CONV:False, (0, 82), ../data/travel_time_series-3361_samples.csv, (1, 83, 4)
Links del dataset desde 0 - 82 = 83 links
1/1 [==============================] - 0s 18ms/step
=> Pruebas Completadas
BASE-LSTM_e25b256_Robust_in83-4_dt0-82.h5
Parametros de prediccion: 83, (83, 1), Robust, CONV:False, (0, 82), ../data/travel_time_series-3361_samples.csv, (1, 83, 4)
Links del dataset desde 0 - 82 = 83 links
1/1 [==============================] - 0s 14ms/step
=> Pruebas Completadas


In [6]:
preds

,Nro,Stops,Link,Time,Real,Mean_Set,err_mean_set,input_set,err_input_set,input_concat,err_input_concat,model
0,1,Primer Paradero - Hospital,"(39, 45)","(10, 30)",379,447.0,-68.0,366.0,13.0,354.0,25.0,AUTOENCODER-LSTM_e200b256_Robust_in83-4_dt0-82
1,2,Primer Paradero - UNSAAC,"(39, 46)","(10, 30)",457,530.0,-73.0,468.0,-11.0,440.0,17.0,AUTOENCODER-LSTM_e200b256_Robust_in83-4_dt0-82
2,3,Primer Paradero - Hospital,"(39, 45)","(16, 45)",397,444.0,-47.0,433.0,-36.0,423.0,-26.0,AUTOENCODER-LSTM_e200b256_Robust_in83-4_dt0-82
3,4,Primer Paradero - UNSAAC,"(39, 46)","(16, 45)",513,550.0,-37.0,492.0,21.0,481.0,32.0,AUTOENCODER-LSTM_e200b256_Robust_in83-4_dt0-82
4,5,Control - Universidad Andina,"(14, 21)","(12, 45)",374,777.0,-403.0,250.0,124.0,253.0,121.0,AUTOENCODER-LSTM_e200b256_Robust_in83-4_dt0-82
5,6,Control - Universidad Andina,"(14, 21)","(13, 45)",397,481.0,-84.0,253.0,144.0,257.0,140.0,AUTOENCODER-LSTM_e200b256_Robust_in83-4_dt0-82
6,7,Control - Universidad Andina,"(14, 21)","(11, 0)",389,545.0,-156.0,245.0,144.0,247.0,142.0,AUTOENCODER-LSTM_e200b256_Robust_in83-4_dt0-82
7,8,Control - Universidad Andina,"(14, 21)","(14, 30)",371,879.0,-508.0,258.0,113.0,257.0,114.0,AUTOENCODER-LSTM_e200b256_Robust_in83-4_dt0-82
8,9,Universidad Andina - Puerta Universidad UNSAAC,"(22, 45)","(14, 30)",1271,948.0,323.0,1114.0,157.0,1112.0,159.0,AUTOENCODER-LSTM_e200b256_Robust_in83-4_dt0-82
9,10,Universidad Andina - Puerta Universidad UNSAAC,"(22, 45)","(10, 0)",1157,1001.0,156.0,1083.0,74.0,1074.0,83.0,AUTOENCODER-LSTM_e200b256_Robust_in83-4_dt0-82


In [7]:
total_err_input_set = {}
total_err_input_concat = {}
for mod in preds['model'].unique():
    df_tmp = preds[preds['model'] == mod]
    total_err_input_set[mod] = abs(df_tmp['err_input_set']).sum()
    total_err_input_concat[mod] = abs(df_tmp['err_input_concat']).sum()

In [8]:
total_err_input_set

{'AUTOENCODER-LSTM_e200b256_Robust_in83-4_dt0-82': 866.0,
 'BASE-LSTM_e25b256_Robust_in83-4_dt0-82': 1272.0}

In [9]:
total_err_input_concat

{'AUTOENCODER-LSTM_e200b256_Robust_in83-4_dt0-82': 870.0,
 'BASE-LSTM_e25b256_Robust_in83-4_dt0-82': 1230.0}

In [81]:
# leer dataset busstop
df_stops = pd.read_csv('../data/busstops_ida.csv')

# df_stops.info()
def generateLinks(row):
    row['end_stop'] = df_linkref.loc[row.name+1, 'id']
    row['name_end_stop'] = df_linkref.loc[row.name+1, 'name']
    row['linkref'] = str(row['id']) + ':' + str(df_linkref.loc[row.name+1, 'id'])
    return row

df_linkref = df_stops.copy()
df_linkref = df_linkref.head(df_linkref.shape[0]-1).apply(generateLinks, axis=1)
df_linkref = df_linkref.rename(columns={'id':'init_stop'})
df_linkref.drop(columns=['number', 'radio', 'latitude', 'longitude', 'navigation'], inplace=True)
# df_linkref['end_stop'] = df_linkref['end_stop'].values.astype(str)
print(df_linkref.to_string())

    init_stop                       name  end_stop              name_end_stop  linkref
0           0                     inicio        36                primer stop     0:36
1          36                primer stop        37               segundo stop    36:37
2          37               segundo stop        38                   Ccollana    37:38
3          38                   Ccollana        39            Puente Huaccoto    38:39
4          39            Puente Huaccoto        40                      Kayra    39:40
5          40                      Kayra        41                   Forestal    40:41
6          41                   Forestal        42                Grifo Tapia    41:42
7          42                Grifo Tapia        43                      Posta    42:43
8          43                      Posta        44                  Romeritos    43:44
9          44                  Romeritos        45                    Tercero    44:45
10         45                    Tercero   

In [82]:
a = 1157
a/60,a%60

(19.283333333333335, 17)

Mostrar graficas de entrenamiento y mejora de rmse por epocas

In [ ]:
# Definir la ruta de la carpeta que contiene los archivos JSON
ruta_carpeta = '../info_models/'
name_models = ['BASE-LSTM','AUTOENCODER-LSTM','BASE-Conv2DLSTM','AUTOENCODER-Conv2DLSTM']

# Inicializar una lista para almacenar los DataFrames de cada archivo JSON
dataframes = []

# Recorrer los archivos en la carpeta
for archivo in os.listdir(ruta_carpeta):
    if archivo.endswith('.json'):
        # Leer el archivo JSON y cargarlo en un DataFrame
        ruta_archivo = os.path.join(ruta_carpeta, archivo)
        df = pd.read_json(ruta_archivo, lines=True)
        
        # Agregar el DataFrame a la lista
        dataframes.append(df)

# Combinar los DataFrames en uno solo
data = pd.concat(dataframes, ignore_index=True)

# # Mostrar el DataFrame resultante
for k in range(len(name_models)-2):
    
    data_temp = data[data['n_modelo'] == name_models[k]].sort_values('epoch').reset_index()

    # Plotear las columnas Columna1 y Columna2
    plt.scatter(data_temp['epoch'], data_temp['training_time'])
    plt.plot(data_temp['epoch'], data_temp['training_time'], label = str(data_temp['n_modelo'][0] + ' training time'))

    for x, y in zip(data_temp['epoch'], data_temp['training_time']):
        plt.text(x, y, f"({int(x)}, {round(y,2)})", ha='center', va='bottom', fontsize=6)
    plt.xlabel('Epochs')
    plt.ylabel('Seconds')
    plt.title('Epochs vs training time for all models')

    # Mostrar el gráfico
plt.legend()   
plt.show()